<a href="https://colab.research.google.com/github/FedeYB/Laboratorio_1-/blob/main/Laboratorio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conexión con el drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Librerias


In [3]:
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np

#Limpieza del csv

In [4]:
ruta_de_acceso = '/content/drive/MyDrive/UPSO/UPSO/Segundo año/Segundo cuatrimestre/Elemento de aprendizaje de maquina y big data/Clase 3/Dataset de ventas2'

archivos = os.listdir(ruta_de_acceso)

dataframes = {}# este diccionario sirve para dsp guardar el nombre del mes con el dataframe del mes
               # ej: 'Enero': Df_enero

for archivo in archivos:
  if archivo.endswith('.csv'):
    #mes = archivo.split('_')[3] # te sepeara por cada '_' que encuentre, y te devuelve el [3] elemento
    mes = archivo.split('_')[3].split('.')[0] # le agregamos el "split('.')[0]" para eliminar el ".csv" y obtener solo el nombre del mes
    ruta_completa = os.path.join(ruta_de_acceso, archivo) # con esto juntamos la ruta de la cartepeta principalcon el nombre del archivo para obtener la ruta completa
    df = pd.read_csv(ruta_completa)
    df_ventas = df.copy() # hacemos una copia del df para no modificar el original

    # transformo todos lo valores no numericos en NaN en las columnas 'Cantidad Pedida' y 'Precio Unitario'
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors = 'coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors = 'coerce')

    # Elimino todos los NaN y me quedo solo con los valores numericos
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

    df_ventas = df_ventas.dropna()

    # Transformo todos los valores numericos al tipo de dato int y float
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)

    dataframes[mes] = df_ventas # guardamos el mes y el df del mes en el diccionario
    # Habria que ordenar los df dentro de el diccionario


#Funciones

In [5]:
def extraer_datos(datos):
  datos['Fecha de pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de pedido'].dt.hour
  datos['Dia'] = datos['Fecha de pedido'].dt.day
  datos['Dia de la semana'] = datos['Fecha de pedido'].dt.weekday
  return datos


In [6]:
def unificarDataFrames():
  dataframe_unico = pd.DataFrame()
  meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
  for mes in meses:
    dataframe_por_mes = dataframes[mes]
    dataframe_por_mes = extraer_datos(dataframe_por_mes)
    dataframe_unico = dataframe_unico.append(dataframe_por_mes, ignore_index = True)

  return dataframe_unico

In [7]:
meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']

for mes in meses:
  dataframe_mes= dataframes [mes]
  dataframe_ciudades = dataframes [mes]
  dataframe_estados = dataframes [mes]

# Comportamiento de las ventas en los distintos meses.

In [ ]:
# Creo listas para almacenar los totales de ventas y los nombres de los meses
totales_ventas = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
    totales_ventas.append(total_ventas_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ventas,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

# Optimizacion de la publicidad y patron de ventas por hora.

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites
    Ventas_por_hora = dataframe_mes.groupby('Hora')['Cantidad Pedida'].count()



    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=Horas,
        y=Ventas_por_hora,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()
horas_analizadas = [19, 12, 16]

for hora in horas_analizadas:
    total_productos_vendidos = []

    for mes in meses:
        dataframe_mes = dataframes[mes]
        dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites

        ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

        # Calcular la cantidad total de productos vendidos para la hora actual
        total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].count())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=meses,
        y=total_productos_vendidos,
        mode='lines',
        name=f'{hora} horas',
        line=dict(width=2, shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Meses", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

# Distribucion de ventas por ubicacion.

In [ ]:
dataframe_ciudades["Ciudad"] = dataframe_ciudades['Dirección de Envio'].str.split(',').str[1].str.strip()
dataframe_estados["Estado"] = dataframe_estados['Dirección de Envio'].str.split(',').str[2].str.strip()
dataframe_estados["Estado"] =dataframe_estados["Estado"].str.split(" ").str[0].str.strip()


fig = go.Figure()


ciudades=dataframe_ciudades['Ciudad'].tolist()
lista_ciudades = list(set(ciudades))


for ciudad in lista_ciudades:

  total_productos_vendidos = []

  for mes in meses:
    dataframe_ciudades = dataframes [mes]
    dataframe_ciudades["Ciudad"] = dataframe_ciudades['Dirección de Envio'].str.split(',').str[1].str.strip()

    dataframe_mes=dataframes[mes]
    ventas_ciudad=dataframe_ciudades[dataframe_ciudades["Ciudad"] == ciudad]
    total_productos_vendidos.append(ventas_ciudad['Cantidad Pedida'].sum())

  fig.add_trace(go.Scatter(
        x=meses,
        y=total_productos_vendidos,
        mode='lines',
        name=ciudad,
        line=dict(width=2, shape='spline')
    ))

  fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

In [ ]:
fig = go.Figure()

estados=dataframe_estados['Estado'].tolist()
lista_estados = list(set(estados))

for estado in lista_estados:

  total_productos_vendidos_E = []

  for mes in meses:
    dataframe_estados = dataframes [mes]
    dataframe_estados["Estado"] = dataframe_estados['Dirección de Envio'].str.split(',').str[2].str.strip()
    dataframe_estados["Estado"] =dataframe_estados["Estado"].str.split(" ").str[0].str.strip()
    dataframe_mes=dataframes[mes]
    ventas_estados=dataframe_estados[dataframe_estados["Estado"] == estado]
    total_productos_vendidos_E.append(ventas_estados['Cantidad Pedida'].sum())

  fig.add_trace(go.Scatter(
        x=meses,
        y=total_productos_vendidos_E,
        mode='lines',
        name=estado,
        line=dict(width=2, shape='spline')
    ))

  fig.update_layout(
    title="Unidades vendidas por estados",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

#Tendencia ventas



###Dividido en meses


####¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días en los diferentes meses?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Dias = list(range(1,32))
color_index = 0
colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#FF5733', '#33FF57']

fig = go.Figure()

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes)
  Ventas_por_dia = dataframe_mes.groupby(['Dia'])['Cantidad Pedida'].sum()

  max_valor = Ventas_por_dia.max()

  markers = []

  for val in Ventas_por_dia:
    if val == max_valor:
      markers.append(val)
    else:
      markers.append(None)

  color_linea = colores[color_index]

  fig.add_trace(go.Scatter(

    x=Dias,
    y=Ventas_por_dia,
    mode='lines',
    name=mes,
    line=dict(shape='spline', smoothing=0.8, width=2, color=color_linea)
    )
  )

  fig.add_trace(go.Scatter(
        x=Dias,
        y=markers,
        mode='markers',
        name=mes,
        marker=dict(symbol='circle', size=10, opacity=1, color=color_linea),
        showlegend=False
    ))
  color_index += 1

fig.update_layout(

    title="Unidades vendidas por dia en el mes",
    xaxis=dict(title="Dias del mes", tickvals=Dias),
    yaxis=dict(title="Unidades vendidas", side="left"))


fig.show()

####¿Se observa alguna diferencia en la venta cada producto a lo largo de los meses?

In [8]:
Dias = list(range(1,32))

dataframe_unidos = unificarDataFrames()

tabla_pivote = dataframe_unidos.pivot_table(index='Dia', columns='Producto', values='Cantidad Pedida', aggfunc='sum')

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_unico = dataframe_unico.append(dataframe_por_mes, ignore_index = True)
<ipython-input-6-ba1ab290ec5b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_unico = dataframe_unico.append(dataframe_por_mes, ignore_index = True)
<ipython-input-6-ba1ab290ec5b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_unico = dataframe_unico.append(dataframe_por_mes, ignore_index = True)
<ipython-input-6-ba1ab290ec5b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_unico = dataframe_unico.append(dataframe_por_mes, ignore_index = True)


In [9]:
fig = go.Figure()

for producto in tabla_pivote.columns:
    fig.add_trace(go.Scatter(
        x=Dias,
        y=tabla_pivote[producto],
        mode='lines',
        name=producto,
        line=dict(shape='spline', smoothing=0.8, width=2)
    ))

fig.update_layout(

    title="Productos vendidos por dia en el mes",
    xaxis=dict(title="Dias del mes", tickvals=Dias),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()

####¿Se observa alguna diferencia en la rentabilidad de cada producto a lo largo de los dias de los meses?

In [10]:
Dias = list(range(1,32))

dataframe_unidos = unificarDataFrames()

tabla_pivote = dataframe_unidos.pivot_table(index='Dia', columns='Producto', values='Cantidad Pedida', aggfunc='sum')

tabla_precio_unitario = dataframe_unidos.pivot_table(index='Dia', columns='Producto', values='Precio Unitario', aggfunc='first')

tabla_ventas = tabla_pivote * tabla_precio_unitario

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

In [11]:
fig = go.Figure()

for producto in tabla_ventas.columns:
    fig.add_trace(go.Scatter(
        x=Dias,
        y=tabla_ventas[producto],
        mode='lines',
        name=producto,
        line=dict(shape='spline', smoothing=0.8, width=2)
    ))

fig.update_layout(

    title="Ganancias por productos vendidos a lo largo de los mes",
    xaxis=dict(title="Dias del mes", tickvals=Dias),
    yaxis=dict(title="Ganancias", side="left"))

fig.show()

###Dividido en semanas

####¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Dias_de_la_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
markers = []
colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#FF5733', '#33FF57']
color_index = 0
fig = go.Figure()

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes)
  Ventas_por_dia_de_semana = dataframe_mes.groupby(['Dia de la semana'])['Cantidad Pedida'].sum()

  max_valor = Ventas_por_dia_de_semana.max()

  markers = []

  for val in Ventas_por_dia_de_semana:
    if val == max_valor:
      markers.append(val)
    else:
      markers.append(None)

  color_linea = colores[color_index]

  fig.add_trace(go.Scatter(

      x=Dias_de_la_semana,
      y=Ventas_por_dia_de_semana,
      mode='lines',
      name=mes,
      line=dict(shape='spline', smoothing=0.8, width=2, color=color_linea),
      text=markers
      ))

  fig.add_trace(go.Scatter(
        x=Dias_de_la_semana,
        y=markers,
        mode='markers',
        name=mes,
        marker=dict(symbol='circle', size=10, opacity=1, color=color_linea),
        showlegend=False
    ))

  color_index += 1

fig.update_layout(

    title="Unidades vendidas por dia de semana",
    xaxis=dict(title="Dias de la semana", tickvals=Dias_de_la_semana),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()


####¿Se observa alguna diferencia significativa en las ventas entre los días laborables y los fines de semana?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Dias_de_la_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
mediana_dias = []
Lunes = []
Martes = []
Miercoles = []
Jueves = []
Viernes = []
Sabado = []
Domingo = []

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes)
  Ventas_por_dia_de_semana = dataframe_mes.groupby(['Dia de la semana'])['Cantidad Pedida'].sum()
  Lunes.append(Ventas_por_dia_de_semana[0])
  Martes.append(Ventas_por_dia_de_semana[1])
  Miercoles.append(Ventas_por_dia_de_semana[2])
  Jueves.append(Ventas_por_dia_de_semana[3])
  Viernes.append(Ventas_por_dia_de_semana[4])
  Sabado.append(Ventas_por_dia_de_semana[5])
  Domingo.append(Ventas_por_dia_de_semana[6])

mediana_dias.append(np.median(Lunes))
mediana_dias.append(np.median(Martes))
mediana_dias.append(np.median(Miercoles))
mediana_dias.append(np.median(Jueves))
mediana_dias.append(np.median(Viernes))
mediana_dias.append(np.median(Sabado))
mediana_dias.append(np.median(Domingo))

colores = ['blue', 'orange', 'green', 'red', 'purple', 'burlywood', 'pink']

#Creo el objeto figura
fig = go.Figure()

#Agrego grafico en mi objeto figura
fig.add_trace(go.Bar(
    x = Dias_de_la_semana,
    y = mediana_dias,
    name = 'Unidades Vendidas',
    marker = dict(color = colores),
    )
)

fig.update_layout(
    title = 'Mediana de las unidades vendidas por dia de la semana',
    xaxis = dict(title = 'Dias de la semana'),
    yaxis = dict(title = 'Mediana de las unidades vendidas', side = 'left')
)


####¿Se observa alguna diferencia en la venta cada producto a lo largo de los dias de la semana?

In [12]:
Dias_de_la_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']

dataframe_unidos = unificarDataFrames()

tabla_pivote = dataframe_unidos.pivot_table(index='Dia de la semana', columns='Producto', values='Cantidad Pedida', aggfunc='sum')

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

In [13]:
fig = go.Figure()

for producto in tabla_pivote.columns:
    fig.add_trace(go.Scatter(
        x=Dias_de_la_semana,
        y=tabla_pivote[producto],
        mode='lines',
        name=producto,
        line=dict(shape='spline', smoothing=0.8, width=2)
    ))

fig.update_layout(

    title="Productos vendidos por dia de la semana",
    xaxis=dict(title="Dias de la semana", tickvals=Dias_de_la_semana),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()

In [14]:
index_dia = 0

for dia, fila in tabla_pivote.iterrows():
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=tabla_pivote.columns,
        y=fila,
        name='Unidades Vendidas',
        text=fila,
        textposition='auto'
    ))


    fig.update_layout(
        title=f'Unidades vendidas el {Dias_de_la_semana[index_dia]}',
        xaxis=dict(title='Producto'),
        yaxis=dict(title='Mediana de las unidades vendidas', side='left')
    )

    index_dia += 1

    fig.show()

####¿Se observa alguna diferencia en la rentabilidad de cada producto a lo largo de la semana?

In [15]:
Dias_de_la_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']

dataframe_unidos = unificarDataFrames()

tabla_pivote = dataframe_unidos.pivot_table(index='Dia de la semana', columns='Producto', values='Cantidad Pedida', aggfunc='sum')

tabla_precio_unitario = dataframe_unidos.pivot_table(index='Dia de la semana', columns='Producto', values='Precio Unitario', aggfunc='first')

tabla_ventas = tabla_pivote * tabla_precio_unitario

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-6-ba1ab290ec5b>:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

In [16]:
fig = go.Figure()

for producto in tabla_ventas.columns:
    fig.add_trace(go.Scatter(
        x=Dias_de_la_semana,
        y=tabla_ventas[producto],
        mode='lines',
        name=producto,
        line=dict(shape='spline', smoothing=0.8, width=2)
    ))

fig.update_layout(

    title="Ganancias de productos vendidos por dia de la semana",
    xaxis=dict(title="Dias de la semana", tickvals=Dias_de_la_semana),
    yaxis=dict(title="Ingresos", side="left"))

fig.show()